In [3]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels/')
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.particlesLoader import getParticlesFromSLHA

pd.options.mode.chained_assignment = None #Disable copy warnings

In [2]:
BSMlist = getParticlesFromSLHA('../../slhaFiles_all/run_01_mzp_3684_mchi_1440_msd_1842.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, zp, ghzp, ghzp~, sd, chi]


In [4]:
slhaFolder = '../../slhaFiles_all'
resultsFolder = '../../data/2mdm_results_all/'
parametersFile = "/home/yoxara/2MDM/notebooks/SmodelS/parameters_2mdm.ini"
!chmod -R +rwx {slhaFolder}

In [5]:
!runSModelS.py -p {parametersFile} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadBinaryFile() in 526: loading binary db file /home/yoxara/smodels/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 533: Loaded database from /home/yoxara/smodels/smodels-database/db3.pcl in 3.3 secs.
INFO in modelTester.loadDatabaseResults() in 498: Including non-validated results
INFO in modelTester.testPoints() in 358: Running SModelS for 500 files with a single process. Messages will be redirected to smodels.log


In [7]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    dataF = dataF.replace('None', '-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
#             if res['r'] == None:
#                 res['r'] = np.nan
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [8]:
print(len(data))

500


In [10]:
#Convert data to flat DataFrame:
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,...,ExptRes.result3.l_SM,ExptRes.result0.likelihood,ExptRes.result0.l_max,ExptRes.result0.l_SM,ExptRes.result1.likelihood,ExptRes.result1.l_max,ExptRes.result1.l_SM,ExptRes.result2.likelihood,ExptRes.result2.l_max,ExptRes.result2.l_SM
0,run_27_mzp_2654_mchi_785_msd_1327.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7494.501000,0.0,7501.106000,0.000000,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,run_130_mzp_2692_mchi_333_msd_1346.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",3908.261000,0.0,3912.839000,0.000000,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,run_93_zp_3677.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.000000,0.0,6.712587,3.104914,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,run_220_mzp_1135_mchi_301_msd_568.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",16.934000,0.0,1686.254000,277.462700,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,run_247_zp_3300.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.000000,0.0,13.426990,11.994010,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,run_148_zp_3053.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.000000,0.0,24.930150,22.431860,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,run_156_mzp_1788_mchi_745_msd_894.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.215200,0.0,176.792000,0.000000,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,run_25_zp_2722.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.000000,0.0,66.360640,0.000000,0.0,5.0,0.2,1,...,0.007032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,run_36_mzp_1788_mchi_233_msd_894.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.489784,0.0,103.609300,0.000000,0.0,5.0,0.2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 52])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['ZPRIME'].items()])
    extparDict.update(dict([[str(key+len(extparDict)),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    extparDict.update(dict([[str(key+len(extparDict)),val] for key,val in slha.blocks['FRBLOCK'].items()]))
    
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 52])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [20]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [21]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [22]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns name

Final number of data points: 500


In [23]:
dataDF

,filename,mass.9000006,mass.9900026,mass.9900032,mass.9000002,mass.9000003,mass.9000004,mass.9000005,width.9000002,width.9000003,...,ExptRes.result3.l_SM,ExptRes.result0.likelihood,ExptRes.result0.l_max,ExptRes.result0.l_SM,ExptRes.result1.likelihood,ExptRes.result1.l_max,ExptRes.result1.l_SM,ExptRes.result2.likelihood,ExptRes.result2.l_max,ExptRes.result2.l_SM
0,run_27_mzp_2654_mchi_785_msd_1327.slha,784.8223,1327.0670,2654.135,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,run_130_mzp_2692_mchi_333_msd_1346.slha,332.6893,1346.1960,2692.392,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,run_93_zp_3677.slha,1235.0080,1838.5560,3677.111,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,run_220_mzp_1135_mchi_301_msd_568.slha,300.9204,567.5892,1135.178,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,run_247_zp_3300.slha,636.7754,1649.8760,3299.752,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,run_148_zp_3053.slha,580.8612,1526.3100,3052.620,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,run_156_mzp_1788_mchi_745_msd_894.slha,745.4011,893.8177,1787.635,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,run_25_zp_2722.slha,764.6948,1361.1690,2722.339,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,0.007032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,run_36_mzp_1788_mchi_233_msd_894.slha,233.0672,893.9976,1787.995,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Save DataFrame to pickle file:
dataDF.to_pickle('../../data/2mdm_results_all/results.pcl')
print(len(dataDF))
dataDF

500


,filename,mass.9000006,mass.9900026,mass.9900032,mass.9000002,mass.9000003,mass.9000004,mass.9000005,width.9000002,width.9000003,...,ExptRes.result3.l_SM,ExptRes.result0.likelihood,ExptRes.result0.l_max,ExptRes.result0.l_SM,ExptRes.result1.likelihood,ExptRes.result1.l_max,ExptRes.result1.l_SM,ExptRes.result2.likelihood,ExptRes.result2.l_max,ExptRes.result2.l_SM
0,run_27_mzp_2654_mchi_785_msd_1327.slha,784.8223,1327.0670,2654.135,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,run_130_mzp_2692_mchi_333_msd_1346.slha,332.6893,1346.1960,2692.392,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,run_93_zp_3677.slha,1235.0080,1838.5560,3677.111,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,run_220_mzp_1135_mchi_301_msd_568.slha,300.9204,567.5892,1135.178,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,run_247_zp_3300.slha,636.7754,1649.8760,3299.752,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,run_148_zp_3053.slha,580.8612,1526.3100,3052.620,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,run_156_mzp_1788_mchi_745_msd_894.slha,745.4011,893.8177,1787.635,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,run_25_zp_2722.slha,764.6948,1361.1690,2722.339,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,0.007032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,run_36_mzp_1788_mchi_233_msd_894.slha,233.0672,893.9976,1787.995,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
print(dataDF.columns)


Index(['filename', 'mass.9000006', 'mass.9900026', 'mass.9900032',
       'mass.9000002', 'mass.9000003', 'mass.9000004', 'mass.9000005',
       'width.9000002', 'width.9000003',
       ...
       'ExptRes.result3.l_SM', 'ExptRes.result0.likelihood',
       'ExptRes.result0.l_max', 'ExptRes.result0.l_SM',
       'ExptRes.result1.likelihood', 'ExptRes.result1.l_max',
       'ExptRes.result1.l_SM', 'ExptRes.result2.likelihood',
       'ExptRes.result2.l_max', 'ExptRes.result2.l_SM'],
      dtype='object', length=147)
